In [3]:
!julia -p 2

LoadError: syntax: extra token "2" after end of expression
while loading In[3], in expression starting on line 1

In [1]:
# based on https://github.com/chkwon/TrafficAssignment.jl

include("load_network_uni-class.jl")

using Graphs

function create_graph(start_node, end_node)
    @assert Base.length(start_node)==Base.length(end_node)

    no_node = max(maximum(start_node), maximum(end_node))
    no_arc = Base.length(start_node)

    graph = simple_inclist(no_node)
    for i=1:no_arc
        add_edge!(graph, start_node[i], end_node[i])
    end
    return graph
end

function get_vector(state, origin, destination, link_dic)
    current = destination
    parent = -1
    x = zeros(Int, maximum(link_dic))

    while parent != origin
        parent = state.parents[current]

        link_idx = link_dic[parent,current]

        if link_idx != 0
            x[link_idx] = 1
        end

        current = parent
    end

    return x
end

ta_data = load_ta_network("Sioux Falls");

# ta_data.travel_demand;

# unpacking data from ta_data
network_name = ta_data.network_name

number_of_zones = ta_data.number_of_zones
number_of_nodes = ta_data.number_of_nodes
first_thru_node = ta_data.first_thru_node
number_of_links = ta_data.number_of_links

start_node = ta_data.start_node
end_node = ta_data.end_node
capacity = ta_data.capacity
link_length = ta_data.link_length

free_flow_time = ta_data.free_flow_time
B = ta_data.B
power = ta_data.power
speed_limit = ta_data.speed_limit
toll = ta_data.toll
link_type = ta_data.link_type
number_of_zones = ta_data.number_of_zones
total_od_flow = ta_data.total_od_flow
travel_demand = ta_data.travel_demand
od_pairs = ta_data.od_pairs

toll_factor = ta_data.toll_factor
distance_factor = ta_data.distance_factor

best_objective = ta_data.best_objective

travel_demand

# preparing a graph
graph = create_graph(start_node, end_node)
link_dic = sparse(start_node, end_node, 1:number_of_links);

function BPR(x)
    bpr = similar(x)
    for i=1:length(bpr)
        bpr[i] = free_flow_time[i] * ( 1.0 + B[i] * (x[i]/capacity[i])^power[i] )
    end
    return bpr
end

function all_or_nothing(travel_time)
    state = []
    path = []
    x = zeros(size(start_node))

    for r=1:size(travel_demand)[1]
        # for each origin node r, find shortest paths to all destination nodes
        state = dijkstra_shortest_paths(graph, travel_time, r)

        for s=1:size(travel_demand)[2]
            # for each destination node s, find the shortest-path vector
            # load travel demand
            x = x + travel_demand[r,s] * get_vector(state, r, s, link_dic)
        end
    end

    return x
end

# Finding a starting feasible solution
travel_time = BPR(zeros(number_of_links))
xl = all_or_nothing(travel_time);

max_iter_no = 3e3
l = 1
#average_excess_cost = 1
tol = 1e-6

while l < max_iter_no
    l += 1
    
    xl_old = xl
    
    # Finding yl
    travel_time = BPR(xl)
    
    yl = all_or_nothing(travel_time)
    
    assert(yl != xl)
    
    xl = xl + (yl - xl)/l
    
    xl_new = xl
    
    relative_gap = norm(xl_new - xl_old, 1) / norm(xl_new, 1)

    if l % 2000 == 0
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
    end

    if relative_gap < tol 
        print("l = $l------------------------------------------------\n")
        print("relative_gap is $relative_gap\n")
        break
    end
    
end

flows = Dict{(Int64,Int64),Float64}()

for i = 1:length(ta_data.start_node)
    key = (ta_data.start_node[i], ta_data.end_node[i])
    flows[key] = xl[i]
end

flows